<h2
    style="font-size:20px; font-family:'Helvetica Neue'" align="right"> Viktor Trokhymenko 
</h2>
<h3 
    style="font-size:16px; font-family:'Helvetica Neue'" align="right"> @vtrokhymenko
</h3>

In [53]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
np.random.seed(42)
import re
from pprint import pprint

import nltk
from nltk.corpus import stopwords

#%load_ext autoreload
#%autoreload 2

#### пока для примера беру лемматиатор от `pymorphy2` & `pymystem3`

* pymorphy2

In [3]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
#затестим лемматизатор
morph.parse('приехали')[0].normal_form

'приехать'

* pymystem3 (возьмем за основу)

In [5]:
from pymystem3 import Mystem

In [6]:
m=Mystem()

Installing mystem to /Users/vtrokhymenko/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


In [11]:
m.lemmatize('машиной')[0]

'машина'

#### добавим дополнительное(ые) стоп-слово(а), которое(ые) удалось заметить, но эти стоп-слова нужно по мере необходимости дополнять

In [19]:
stopword_ru=stopwords.words('russian')
print(len(stopword_ru))
print(stopword_ru)

151
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 

In [20]:
stopword_ru.append('м')
print(len(stopword_ru))
print(stopword_ru)

152
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 

#### читаем данные

In [21]:
#https://github.com/buriy/russian-nlp-datasets/releases
df = pd.read_csv('data/novayagazeta.ru.txt', sep='\t', encoding='utf-8', header=None)
df.columns = ['id', 'url', 'title', 'text']

print('before filter duplicates: ',df.shape)
df = df.drop_duplicates('url')
print('after filter duplicates: ',df.shape)
df.head()

before filter duplicates:  (14491, 4)
after filter duplicates:  (14485, 4)


,id,url,title,text
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са..."
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России..."


In [14]:
df.loc[4].text

'1. Белоруссия и Казахстан, с которыми у России таможенный союз (что означает отсутствие таможенного контроля на границе), не поддержали это решение. Следовательно, те же товары, но гораздо дороже, будут приходить в Россию как контрабанда. Думаю, никто в этом и не сомневается. По официальным опубликованным данным за пять месяцев 2014 года, белорусских лимонов в Россию поставлено на $1,7 млн, белорусских бананов — на $183 000, белорусских ананасов — на $37 000, белорусских кокосовых и миндальных орехов — почти на $1,8 млн, белорусских мидий — на $792 000, белорусских осьминогов — на $449 000.\\n2. Все сделанные в последнее время и уже опубликованные расчеты показывают, что в результате этих «ответных санкций» вырастут цены на продовольствие в наших магазинах. Известно, что на поставки из стран, в отношении которых наложен запрет, приходится порядка 20% от общего объема потребления. Одной из главных причин роста цен станет сокращение поставок мяса, тем более что цены на мясо на мировом р

#### ф-я для чистки текста: 
* от мусора и 
* удаляем стоп-слова

In [22]:
def clean_text(text): 
    
    #убираем мусор
    text = nltk.re.sub('-\s\r\n\s+|-\s\r\n|\r\n|[«»]|[><]', ' ', text.lower())    
    text = nltk.re.sub('[0-9]|[.,:;_%©?*!@#№$^•·&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-|–|—|', ' ', text)
    text = re.sub(r"\d+", ' ', text, flags=re.UNICODE)
    text = re.sub('\’|\”|‘', ' ', text)
    text = re.sub(r'\\n', '', text)
    text = re.sub('ё', 'е', text)
    
    #убираем стоп-слова
    words = text.split()
    words = [w for w in words if not w in stopword_ru]
    
    #джойним слова
    return ' '.join(words)

clean_text(df.loc[4].text)

'белоруссия казахстан которыми россии таможенный союз означает отсутствие таможенного контроля границе поддержали это решение следовательно те товары гораздо дороже будут приходить россию контрабанда думаю никто сомневается официальным опубликованным данным пять месяцев года белорусских лимонов россию поставлено млн белорусских бананов белорусских ананасов белорусских кокосовых миндальных орехов млн белорусских мидий белорусских осьминогов сделанные последнее время опубликованные расчеты показывают результате этих ответных санкций вырастут цены продовольствие наших магазинах известно поставки стран отношении которых наложен запрет приходится порядка общего объема потребления одной главных причин роста цен станет сокращение поставок мяса цены мясо мировом рынке растут результате рост цен ближайшие полгода составит решение запрете ввоз продуктов принято условиях российская экономика осени минувшего года резко тормозит итогам первой половины года вошла рецессию т е начался экономический с

In [23]:
%%time

df['text_clear']=df.text.apply(clean_text)

CPU times: user 25.5 s, sys: 323 ms, total: 25.8 s
Wall time: 26.2 s


In [24]:
df.head()

,id,url,title,text,text_clear
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",предсказывал страницах также сайте новой иван ...
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...,август освобожденные заключенные площади оперы...
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...,август россии время трагических годовщин десят...
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...,избирательном округе районы выхино жулебино ря...
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России...",белоруссия казахстан которыми россии таможенны...


In [26]:
df.loc[np.random.randint(low=0, high=df.shape[0])].text_clear

'порошенко большая часть российских войск покинула восток украины президент украины петр порошенко заявил большая часть российских войск находившихся востоке украины вернулась территорию россии сообщает reuters согласно последней информации которую получил спецслужб российских войск вернулись россию сказал украинский президент совещании правительством мнению порошенко это увеличивает шансы мирного урегулирования ситуации востоке страны напомним сентября украинские власти сепаратисты договорились прекращении огня донецкой луганской областях ранее петр порошенко заявил вторжении российских вооруженных сил территорию украины однако власти россии информацию неоднократно опровергали'

#### применяем лемматизатор

In [27]:
%%time

#df['text_clear_lemma'] = df['text_clear'].astype(str).apply(lambda x: [morph.parse(i)[0].normal_form for i in x.split()])
df['text_clear_lemma'] = df['text_clear'].astype(str).apply(lambda x: [m.lemmatize(i)[0] for i in x.split()])

CPU times: user 3min 3s, sys: 1min 2s, total: 4min 6s
Wall time: 10min 53s


In [28]:
df.head()

,id,url,title,text,text_clear,text_clear_lemma
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",предсказывал страницах также сайте новой иван ...,"[предсказывать, страница, также, сайт, новый, ..."
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...,август освобожденные заключенные площади оперы...,"[август, освобождать, заключенная, площадь, оп..."
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...,август россии время трагических годовщин десят...,"[август, россия, время, трагический, годовщина..."
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...,избирательном округе районы выхино жулебино ря...,"[избирательный, округ, район, выхино, жулебино..."
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России...",белоруссия казахстан которыми россии таможенны...,"[белоруссия, казахстан, который, россия, тамож..."


In [29]:
df.to_csv('data/novayagazeta_clear_pymystem3.csv',index=False)

In [31]:
print(df.loc[np.random.randint(low=0, high=df.shape[0])].text_clear_lemma)

['спецпредставитель', 'президент', 'россия', 'ближний', 'восток', 'страна', 'африка', 'замминистра', 'иностранный', 'дело', 'михаил', 'богданов', 'встречаться', 'один', 'лидер', 'сирийский', 'оппозиция', 'бывший', 'председатель', 'национальный', 'коалиция', 'сила', 'сирийский', 'революция', 'оппозиция', 'ахмед', 'аль', 'джарба', 'сообщать', 'прес', 'служба', 'мид', 'рф', 'читать', 'также', 'россия', 'настойчиво', 'стремиться', 'становиться', 'военный', 'союзник', 'сша', 'сирийский', 'кризис', 'американский', 'элита', 'пока', 'накануне', 'выбор', 'откладывать', 'вопросвстреча', 'проходить', 'накануне', 'каир', 'сообщение', 'говориться', 'сторона', 'обсуждать', 'ситуация', 'сирия', 'необходимость', 'продвижение', 'политический', 'урегулирование', 'сирийский', 'кризис', 'основа', 'женевский', 'коммюнике', 'июнь', 'год', 'богданов', 'также', 'объяснять', 'оппозиционер', 'цель', 'российский', 'военный', 'операция', 'страна', 'который', 'утверждаться', 'направлять', 'оказание', 'эффективный'

## LDA

In [50]:
from gensim import corpora, models

* присваиваем каждому токену уникальный идентификатор, то есть `id`

In [33]:
%%time

dictionary = corpora.Dictionary(df['text_clear_lemma'].values)

CPU times: user 6.18 s, sys: 117 ms, total: 6.3 s
Wall time: 6.31 s


In [34]:
print(len(dictionary.token2id))
dictionary.token2id

#не особо понимаю, почему "ё" есть в словах, если я делал в очистке ф-и такое `re.sub('ё', 'е', text)` и отдельно работает

106171


{'автопром': 0,
 'аккуратно': 1,
 'аккуратный': 2,
 'акселерат': 3,
 'активный': 4,
 'амнистировать': 5,
 'аналог': 6,
 'аналогия': 7,
 'анекдот': 8,
 'антисанкция': 9,
 'бада': 10,
 'бедно': 11,
 'безлактозный': 12,
 'безусловно': 13,
 'белорусский': 14,
 'белье': 15,
 'бесчеловечный': 16,
 'биологически': 17,
 'болид': 18,
 'большой': 19,
 'бородатый': 20,
 'буш': 21,
 'быстрый': 22,
 'быть': 23,
 'бюджет': 24,
 'вводить': 25,
 'ведомство': 26,
 'вещь': 27,
 'виагра': 28,
 'видимо': 29,
 'возвращение': 30,
 'вредный': 31,
 'вспомнить': 32,
 'вто': 33,
 'второй': 34,
 'вывешивать': 35,
 'выводить': 36,
 'выкапывать': 37,
 'выпадение': 38,
 'выясняться': 39,
 'геннадий': 40,
 'гибридный': 41,
 'гмо': 42,
 'год': 43,
 'голова': 44,
 'голос': 45,
 'горох': 46,
 'госаппарат': 47,
 'гражданин': 48,
 'давать': 49,
 'два': 50,
 'действие': 51,
 'действительность': 52,
 'дело': 53,
 'диабет': 54,
 'добавка': 55,
 'должно': 56,
 'должный': 57,
 'доллар': 58,
 'другой': 59,
 'еда': 60,
 'женски

* делаем список преобразованных текстов или же "To actually convert tokenized documents to vectors"

In [35]:
%%time

doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['text_clear_lemma'].values]

CPU times: user 4.92 s, sys: 254 ms, total: 5.17 s
Wall time: 5.25 s


In [49]:
print(doc_term_matrix[123])

[(43, 5), (50, 3), (53, 1), (64, 2), (68, 1), (83, 1), (174, 1), (183, 1), (192, 1), (223, 1), (313, 1), (388, 1), (516, 1), (581, 1), (601, 1), (614, 1), (659, 1), (695, 1), (807, 1), (903, 1), (927, 1), (945, 1), (1129, 1), (1133, 1), (1161, 1), (1163, 2), (1212, 3), (1228, 1), (1496, 1), (1514, 1), (1575, 2), (1643, 2), (1644, 1), (1750, 1), (1786, 1), (1789, 1), (1799, 1), (1916, 1), (1941, 1), (1969, 1), (2075, 1), (2128, 1), (2335, 1), (2613, 1), (2615, 1), (2643, 1), (2717, 1), (2727, 1), (2744, 1), (2804, 1), (2809, 1), (2854, 2), (2949, 3), (3002, 1), (3024, 1), (3180, 1), (3222, 1), (3464, 1), (3805, 1), (3907, 1), (3909, 1), (3922, 1), (3942, 1), (3948, 1), (4162, 1), (4629, 2), (4637, 1), (4668, 1), (4670, 1), (4732, 3), (4758, 1), (4804, 1), (4806, 2), (5018, 3), (5696, 1), (6622, 2), (6792, 1), (8120, 5), (8121, 1), (8122, 2), (8123, 3), (8124, 4), (8125, 1), (8126, 1), (8127, 1), (8128, 1)]


In [38]:
print(df.text_clear_lemma[123])
print(doc_term_matrix[123])

['суд', 'приговаривать', 'захватчик', 'белгородский', 'банка', 'два', 'год', 'колония', 'поселение', 'октябрьский', 'суд', 'белгород', 'приговаривать', 'александра', 'вдовина', 'захватить', 'офис', 'банка', 'западный', 'два', 'год', 'колония', 'поселение', 'передавать', 'итар', 'тасс', 'судья', 'признавать', 'вдовина', 'виновный', 'самоуправство', 'незаконный', 'лишение', 'свобода', 'два', 'лицо', 'применение', 'оружие', 'срок', 'отбывание', 'наказание', 'случай', 'вдовина', 'отсчитываться', 'апрель', 'год', 'мужчина', 'арестовывать', 'обвинитель', 'запрашивать', 'судно', 'вдовина', 'год', 'месяц', 'колония', 'поселение', 'сторона', 'защита', 'настаивать', 'наказание', 'связанный', 'лишение', 'свобода', 'удовлетворять', 'вынести', 'приговор', 'процесс', 'дело', 'захват', 'офис', 'банка', 'проходить', 'особый', 'порядок', 'исследование', 'доказательство', 'ходатайство', 'обвиняемый', 'такой', 'образ', 'вдовин', 'полностью', 'признавать', 'вина', 'напоминать', 'апрель', 'год', 'отзыв', '

* tf-idf

In [52]:
%%time 

tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

CPU times: user 1.61 s, sys: 461 ms, total: 2.07 s
Wall time: 2.32 s


In [57]:
print(corpus_tfidf[123])

[(43, 0.04337238700035509), (50, 0.07674836319406926), (53, 0.015403375226679566), (64, 0.08984819034754493), (68, 0.03781021746018334), (83, 0.006760829768534811), (174, 0.024500859341925176), (183, 0.022373457326028807), (192, 0.027561586705752116), (223, 0.01869848309160451), (313, 0.044075776604810935), (388, 0.01474066164796024), (516, 0.06072594376064386), (581, 0.029594665941597296), (601, 0.031167763205994917), (614, 0.04499842823068974), (659, 0.0349153467313694), (695, 0.07272117596755573), (807, 0.03149878896954517), (903, 0.026491927000761136), (927, 0.027958115783119148), (945, 0.02501142421190265), (1129, 0.037366240666900946), (1133, 0.06645760492857024), (1161, 0.06534702814027844), (1163, 0.10230920525889563), (1212, 0.12833096440927402), (1228, 0.02612427706051128), (1496, 0.03188574658950131), (1514, 0.03215735426213943), (1575, 0.06309329284239575), (1643, 0.05106693909627816), (1644, 0.036492031736103055), (1750, 0.04145670157776293), (1786, 0.03878753775201464), (

### LDA

* ### gensim

In [41]:
import pyLDAvis.gensim
from gensim.models import CoherenceModel, LdaMulticore

In [44]:
%%time

lda_model = LdaMulticore(doc_term_matrix,
                         num_topics=30,
                         id2word=dictionary,
                         workers=8)

Process ForkPoolWorker-44:
Process ForkPoolWorker-43:
Process ForkPoolWorker-42:
Process ForkPoolWorker-39:
Process ForkPoolWorker-40:
Process ForkPoolWorker-41:
Process ForkPoolWorker-34:
Process ForkPoolWorker-36:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


Traceback (most recent call last):
  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1238, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 5, in <module>
  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 179, in __init__
    minimum_phi_value=minimum_phi_value, per_word_topics=per_word_topics, dtype=dtype
  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamodel.py", line 515, in __init__
    self.update(corpus, chunks_as_numpy=use_numpy)
  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 306, in update
    process_result_queue(force=True)
  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 268, in process_result_queue
    while not result_queue.empty():
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multipro

KeyboardInterrupt: 

Process ForkPoolWorker-54:
Process ForkPoolWorker-55:
Process ForkPoolWorker-49:
Process ForkPoolWorker-52:
Process ForkPoolWorker-53:
Process ForkPoolWorker-50:
Process ForkPoolWorker-47:
Process ForkPoolWorker-45:
Process ForkPoolWorker-51:
Traceback (most recent call last):
Process ForkPoolWorker-46:
Traceback (most recent call last):
Process ForkPoolWorker-48:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Tra

  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiproce

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
%%time
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(ldamodel_fast,
                                doc_term_matrix,
                                dictionary,
                                mds='tsne')

* tf_idf

In [58]:
lda_model_tfidf = models.LdaMulticore(corpus_tfidf,
                                      num_topics=30,
                                      id2word=dictionary,
                                      passes=1,
                                      workers=6)

Process ForkPoolWorker-72:
Process ForkPoolWorker-66:
Process ForkPoolWorker-71:
Process ForkPoolWorker-69:
Process ForkPoolWorker-63:
Process ForkPoolWorker-70:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", li

KeyboardInterrupt: 

Process ForkPoolWorker-87:
Process ForkPoolWorker-77:
Process ForkPoolWorker-88:
Process ForkPoolWorker-78:
Process ForkPoolWorker-85:
Process ForkPoolWorker-76:
Process ForkPoolWorker-86:
Process ForkPoolWorker-89:
Process ForkPoolWorker-79:
Process ForkPoolWorker-74:
Process ForkPoolWorker-73:
Process ForkPoolWorker-84:
Process ForkPoolWorker-82:
Process ForkPoolWorker-83:
Process ForkPoolWorker-80:
Process ForkPoolWorker-75:
Process ForkPoolWorker-81:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-90:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self

  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
KeyboardInterrupt
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, 

  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
KeyboardInterrupt
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:


* ### sklearn

https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/

In [60]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

[Non-Negative Matrix Factorization](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)

[Latent Dirichlet Allocation with online variational Bayes algorithm](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation)



In [47]:
df.head()

,id,url,title,text,text_clear,text_clear_lemma
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",предсказывал страницах также сайте новой иван ...,"[предсказывать, страница, также, сайт, новый, ..."
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...,август освобожденные заключенные площади оперы...,"[август, освобождать, заключенная, площадь, оп..."
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...,август россии время трагических годовщин десят...,"[август, россия, время, трагический, годовщина..."
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...,избирательном округе районы выхино жулебино ря...,"[избирательный, округ, район, выхино, жулебино..."
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России...",белоруссия казахстан которыми россии таможенны...,"[белоруссия, казахстан, который, россия, тамож..."


In [80]:
df['text_clear_lemma_1sentence']=df.text_clear_lemma.apply(lambda x: ' '.join(x))
df.head(1)

,id,url,title,text,text_clear,text_clear_lemma,text_clear_lemma_1sentence
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",предсказывал страницах также сайте новой иван ...,"[предсказывать, страница, также, сайт, новый, ...",предсказывать страница также сайт новый иван с...


In [100]:
vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(df.text_clear_lemma_1sentence)
print(data_vectorized.shape)

(14485, 106070)


In [82]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.17796735074326328 %


In [101]:
tfidf_vectorizer = TfidfVectorizer(**vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df.text_clear_lemma_1sentence)
print(dtm_tfidf.shape)

(14485, 106070)


In [88]:
%%time

# Build LDA Model
lda_model = LatentDirichletAllocation(n_topics=30,               # Number of topics
                                      max_iter=2,                # Max learning iterations
                                      learning_method='online',   
                                      random_state=42,           # Random state
                                      #n_jobs = -1,               # Use all available CPUs
                                     )

lda_output = lda_model.fit_transform(data_vectorized)

CPU times: user 1min 17s, sys: 5.8 s, total: 1min 23s
Wall time: 1min 22s


In [89]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -36423392.01742516
Perplexity:  7029.870624137156
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 2,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': 1,
 'n_topics': 30,
 'perp_tol': 0.1,
 'random_state': 42,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [105]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

AttributeError: module 'pyLDAvis' has no attribute 'sklearn'